In [86]:
import pandas as pd


In [87]:
glassdoor_file = 'resources/glassdoor.csv'
df = pd.read_csv(glassdoor_file)
drop_col = df.columns
drop_col

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn',
       'spark', 'aws', 'excel'],
      dtype='object')

In [88]:
b = ['Job Title', 'Job Description', 'Rating', 'Location', 'Headquarters', 'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors', 'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary', 'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn', 'spark', 'aws', 'excel']
df.drop(axis=1, columns=b, inplace=True)
df.head()

,Salary Estimate,Company Name
0,$53K-$91K (Glassdoor est.),Tecolote Research\n3.8
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System\n3.4
2,$80K-$90K (Glassdoor est.),KnowBe4\n4.8
3,$56K-$97K (Glassdoor est.),PNNL\n3.8
4,$86K-$143K (Glassdoor est.),Affinity Solutions\n2.9


In [89]:
#df['Company Name'].unique

In [90]:
df.drop_duplicates(subset=['Company Name'],inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,Salary Estimate,Company Name
0,$53K-$91K (Glassdoor est.),Tecolote Research\n3.8
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System\n3.4
2,$80K-$90K (Glassdoor est.),KnowBe4\n4.8
3,$56K-$97K (Glassdoor est.),PNNL\n3.8
4,$86K-$143K (Glassdoor est.),Affinity Solutions\n2.9


In [91]:
df1 = df['Company Name'].str.split(pat='\n',expand=True)

In [92]:
df = pd.concat([df, df1],axis=1)
df.head()

,Salary Estimate,Company Name,0,1
0,$53K-$91K (Glassdoor est.),Tecolote Research\n3.8,Tecolote Research,3.8
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System\n3.4,University of Maryland Medical System,3.4
2,$80K-$90K (Glassdoor est.),KnowBe4\n4.8,KnowBe4,4.8
3,$56K-$97K (Glassdoor est.),PNNL\n3.8,PNNL,3.8
4,$86K-$143K (Glassdoor est.),Affinity Solutions\n2.9,Affinity Solutions,2.9


In [93]:
df.drop(axis=1, columns=['Company Name', 1],inplace=True)

In [94]:
df.rename(columns={0:"Company Name"},inplace=True)
df.head()

,Salary Estimate,Company Name
0,$53K-$91K (Glassdoor est.),Tecolote Research
1,$63K-$112K (Glassdoor est.),University of Maryland Medical System
2,$80K-$90K (Glassdoor est.),KnowBe4
3,$56K-$97K (Glassdoor est.),PNNL
4,$86K-$143K (Glassdoor est.),Affinity Solutions


In [95]:
# # Cleaning 
# df2 = df
# df1 = df2['Salary Estimate'].str.rstrip('(Glassdoor est.)')
# df.drop(axis=1, columns=['Salary Estimate'],inplace=True)
# df = pd.concat([df, df1],axis=1)

In [96]:
mask = df['Company Name'].str.contains('LLC').value_counts()

In [97]:
df.drop(mask, inplace=True)

In [98]:
#pd.set_option('display.max_rows',None,'display.max_columns',None)

In [99]:
#Cleaning Company Name Columns
To_remove_lst = [' .LLC', ' INC.', ' LLC', ' ,INC', ' INC', ' Inc.', ', Inc.', ', Inc']
df['Company Name'] = df['Company Name'].str.replace('|'.join(To_remove_lst), '')
To_remove_lst = [' Inc.', ', Inc.', ', Inc', '. Inc']
df['Company Name'] = df['Company Name'].str.replace('|'.join(To_remove_lst), ',')
df['Company Name'] = df['Company Name'].str.upper()


#Cleaning Salary Estimate Column and seperating into upper/lower
temp_df = df['Salary Estimate'].str.split('-',expand=True)
temp_df[0] = temp_df[0].str.replace('(\D+)','')
temp_df[1] = temp_df[1].str.replace('(\D+)','')
temp_df.rename(columns={0: "Lower Salary Range (K)", 1: "Upper Salary Range (K)"},inplace=True)
df = pd.concat([df, temp_df],axis=1)
df.drop(axis=1, columns=['Salary Estimate'], inplace=True)

In [100]:
lower = df['Lower Salary Range (K)']
upper = df['Upper Salary Range (K)']

df['Salary Range'] = lower+',000'+'-'+upper+',000'
df.drop(axis=1, columns=['Lower Salary Range (K)', 'Upper Salary Range (K)'], inplace=True)
df

,Company Name,Salary Range
0,TECOLOTE RESEARCH,"53,000-91,000"
1,UNIVERSITY OF MARYLAND MEDICAL SYSTEM,"63,000-112,000"
2,KNOWBE4,"80,000-90,000"
3,PNNL,"56,000-97,000"
4,AFFINITY SOLUTIONS,"86,000-143,000"
...,...,...
338,SOFTBANK ROBOTICS,"79,000-127,000"
339,SKYSYNC,"120,000-140,000"
340,"DATAMANUSA,","65,000-113,000"
341,23ANDME,"59,000-125,000"


In [101]:
df.to_csv('cleaned_glassdoor.csv', index=False)